In [1]:
import os
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [3]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_root])

    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        return ModelTrainerConfig(
           root_dir=config.root_dir,
           data_path=config.data_path,
           model_ckpt = config.model_ckpt,
           num_train_epochs = params.num_train_epochs,
           warmup_steps = params.warmup_steps,
           per_device_train_batch_size = params.per_device_train_batch_size,
           weight_decay = params.weight_decay,
           logging_steps = params.logging_steps,
           evaluation_strategy = params.evaluation_strategy,
           eval_steps = params.eval_steps,
           save_steps = params.save_steps,
           gradient_accumulation_steps = params.gradient_accumulation_steps
        )
    

In [5]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

In [6]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import TrainingArguments
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        ##loading the data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps, evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps, save_steps=1e6, gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model_pegasus,
            args= trainer_args,
            tokenizer=tokenizer,
            train_dataset=dataset_samsum_pt['test'],
            eval_dataset=dataset_samsum_pt['validation'],
            data_collator=seq2seq_data_collator
        )

        trainer.train()

        ## save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


 

In [5]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
Found existing installation: accelerate 1.4.0
Uninstalling accelerate-1.4.0:
  Successfully uninstalled accelerate-1.4.0
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/37/1f29af63e9c30156a3ed6ebc2754077016577c094f31de7b2631e5d379eb/transformers-4.49.0-py3-none-any.whl.metadata
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/0a/f6/791b9d7eb371a2f385da3b7f1769ced72ead7bf09744637ea2985c83d7ee/accelerate-1.4.0-py3-none-any.whl.metadata
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.

In [7]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-03-11 17:39:49,471: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-11 17:39:49,476: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-11 17:39:49,477: INFO: common: created directory at: artifacts]
[2025-03-11 17:39:49,480: INFO: common: created directory at: artifacts/model_trainer]


: 